У нас 2 тестовые группы и нет повторяющихся пользователей. Чтобы понять какой набор предложений можно считать лучшим, нужно сравнить их между собой. Какие метрики мы будем использовать: 

CR - доля пользователей, которые совершили платеж. 

ARPU - средний доход с каждого пользователя. 

ARPPU - средний доход с каждого заплатившего пользователя.

In [21]:
def get_metrics(df):
    df_total = df.groupby('testgroup', as_index=False).agg({'user_id':'count'}).rename(columns={'user_id':'total_users'}) 
    df_revenue = df.groupby('testgroup', as_index=False).agg({'revenue':'sum'}) 
    df_money = df.query('revenue != 0').groupby('testgroup', as_index=False).agg({'user_id':'count'}).rename(columns={'user_id':'total_customers'}) 
    df_metrics = df_total.merge(df_money, on = 'testgroup').merge(df_revenue, on = 'testgroup')
    df_metrics['CR'] = (df_metrics.total_customers/df_metrics.total_users*100).round(2)
    df_metrics['ARPU'] = (df_metrics.revenue/df_metrics.total_users).round(2)
    df_metrics['ARPPU'] = (df_metrics.revenue/df_metrics.total_customers).round(2)
    df_metrics.loc[3] = ['diff',
                   ((df_metrics.total_users[0] - df_metrics.total_users[1])/df_metrics.total_users[1]*100).round(2), 
                   ((df_metrics.total_customers[0] - df_metrics.total_customers[1])/df_metrics.total_customers[1]*100).round(2),
                   ((df_metrics.revenue[0] - df_metrics.revenue[1])/df_metrics.revenue[1]*100).round(2),
                   ((df_metrics.CR[0] - df_metrics.CR[1])/df_metrics.CR[1]*100).round(2),
                   ((df_metrics.ARPU[0] - df_metrics.ARPU[1])/df_metrics.ARPU[1]*100).round(2),
                   ((df_metrics.ARPPU[0] - df_metrics.ARPPU[1])/df_metrics.ARPPU[1]*100).round(2)]
    return df_metrics

In [22]:
df_metrics = get_metrics(df)
df_metrics

,testgroup,total_users,total_customers,revenue,CR,ARPU,ARPPU
0,a,202103.00,1928.00,5136189.00,0.95,25.41,2664.00
1,b,202667.00,1805.00,5421603.00,0.89,26.75,3003.66
3,diff,-0.28,6.81,-5.26,6.74,-5.01,-11.31


Выводы по тестовой группе: Задействовано на 0.28% больше пользователей. Общая сумма всех покупок больше на 5.26%. На 5% выше средний доход на платящего пользователя. На 11.31% выше средних доход на платящего пользователя.

Вывод по контрольной группе: Число заплативших пользователей выше на 6.8%. Конверция пользователей больше на 6.74%.